In [1]:
import os
import numpy as np
import pandas as pd
# import seaborn as sns
import tensorflow as tf

tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)


import multiprocessing

# from tqdm import tqdm_notebook

import model_testing

# from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')


dataset_loc = "./Churn_Modelling.csv"


# Fix random seed values
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)



2023-06-01 18:02:05.991387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv(dataset_loc, index_col=0)

df['new_column'] = ((df.Balance + df.EstimatedSalary)/df.CreditScore) * (1/df.NumOfProducts)
df['new_column'] = np.where(df.NumOfProducts.isin([3,4]), df['new_column']**2, df['new_column'])

# Converting type of columns to category
df['Geography'] = df['Geography'].astype('category')
df['Gender'] = df['Gender'].astype('category')
df['Surname'] = df['Surname'].astype('category')

# Assigning numerical values and storing it in another columns
df['Geography_new'] = df['Geography'].cat.codes
df['Gender_new'] = df['Gender'].cat.codes
df['Surname_new'] = df['Surname'].cat.codes

df['Geography'] = df['Geography_new']
df['Gender'] = df['Gender_new']
df['Surname'] = df['Surname_new']

df.drop(['CustomerId','Surname_new','Geography_new', 'Gender_new', 'Surname'], axis = 1,inplace=True)
df.sample(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,new_column
RowNumber,,,,,,,,,,,,
6253,596,1,1,32,3,96709.07,2,0,0,41788.37,0,116.189128
4685,623,0,1,43,1,0.00,2,1,1,146379.30,0,117.479374
1732,601,2,0,44,4,0.00,2,1,0,58561.31,0,48.719892


In [3]:
# from sklearn.preprocessing import StandardScaler

# stand_scale = StandardScaler()
# df1 = pd.DataFrame(stand_scale.fit_transform(df.drop(['Exited'],axis=1)), columns=df.columns[:-1])
# df1['Exited'] = df['Exited'].values
# df = df1
# # np.unique(df.Exited)

In [4]:
# Split the DataFrame into features and target variables.
x,y = model_testing.features_target_split(df)

# Split the features and target variables into training and testing sets.
x_train, x_test, y_train, y_test = model_testing.train_test_split(x,y,df)

x_train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,new_column
RowNumber,,,,,,,,,,,
2152,753,0,1,57,7,0.00,1,1,0,159475.08,211.786295
8393,739,1,1,32,3,102128.27,1,1,0,63981.37,224.776238
5007,755,1,0,37,0,113865.23,2,1,1,117396.25,153.153298
4118,561,0,1,37,5,0.00,2,1,0,83093.25,74.058155
7183,692,1,1,49,6,110540.43,2,0,1,107472.99,157.524147


In [5]:
from imblearn.over_sampling import ADASYN
def adasyn_method(x,y,neighbour):
    adap_synt = ADASYN(random_state=42, n_neighbors=neighbour)
    x_new, y_new = adap_synt.fit_resample(x,y)
    
    return x_new, y_new

x_train, y_train = adasyn_method(x_train, y_train, 500)

In [5]:
from sklearn.preprocessing import StandardScaler

stand_scale = StandardScaler()
x_train_1 = pd.DataFrame(stand_scale.fit_transform(x_train), columns=x_train.columns)
# df1['Exited'] = df['Exited'].values
# df = df1
x_train = x_train_1
# np.unique(df.Exited)


stand_scale = StandardScaler()
x_test_1 = pd.DataFrame(stand_scale.fit_transform(x_test), columns=x_test.columns)
# df1['Exited'] = df['Exited'].values
# df = df1
x_test = x_test_1

In [6]:
x_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,new_column
0,1.058568,-0.905933,0.907507,1.715086,0.684723,-1.226059,-0.910256,0.641042,-1.030206,1.042084,-0.131634
1,0.913626,0.300168,0.907507,-0.659935,-0.696202,0.413288,-0.910256,0.641042,-1.030206,-0.623556,-0.126773
2,1.079274,0.300168,-1.101919,-0.184931,-1.731895,0.601687,0.808830,0.641042,0.970680,0.308128,-0.153578
3,-0.929207,-0.905933,0.907507,-0.184931,-0.005739,-1.226059,0.808830,0.641042,-1.030206,-0.290199,-0.183179
4,0.427035,0.300168,0.907507,0.955079,0.339492,0.548318,0.808830,-1.559960,0.970680,0.135042,-0.151942
...,...,...,...,...,...,...,...,...,...,...,...
7995,0.385623,1.506270,-1.101919,-0.374933,0.339492,-1.226059,-0.910256,0.641042,-1.030206,-1.294966,-0.197030
7996,0.634095,1.506270,0.907507,3.330101,-0.005739,-1.226059,0.808830,-1.559960,-1.030206,0.901685,-0.171098
7997,0.168210,-0.905933,-1.101919,-0.184931,1.375185,-0.073747,0.808830,0.641042,0.970680,-0.558088,-0.171753
7998,0.375270,1.506270,0.907507,-0.374933,1.029954,0.394991,0.808830,0.641042,-1.030206,-1.351500,-0.177328


In [88]:


# creating the model
model = tf.keras.models.Sequential()

from keras.layers import Dropout
from tensorflow.keras.layers import Dense

# first hidden layer
model.add(Dense(8,activation = 'relu', input_dim = x_train.shape[1]))

# second hidden layer
model.add(Dense( 8, activation = 'relu'))

# output layer
model.add(Dense( 1,activation = 'sigmoid'))

# Compiling the NN
# binary_crossentropy loss function used when a binary output is expected
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 

model.fit(x_train, y_train, batch_size = 10, epochs = 50)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Make predictions
predictions = model.predict(x_test)



# y_pred_train, y_pred_test, y_pred_test_proba = model_testing.prediction(model, x_train, x_test)

y_pred_train = np.round(model.predict(x_train))
y_pred_test = np.round(model.predict(x_test))
y_pred_test_proba = model.predict(x_test)

# Calculate evaluation metrics
model_evaluation = model_testing.Evaluation(y_train, y_test, y_pred_train, y_pred_test, y_pred_test_proba)

all_metrics,uhuh = model_evaluation.main()
uhuh

Epoch 1/50
1289/1289 [==============================] - 1s 701us/step - loss: 0.4951 - accuracy: 0.7575
Epoch 2/50
1289/1289 [==============================] - 1s 848us/step - loss: 0.4048 - accuracy: 0.8178
Epoch 3/50
1289/1289 [==============================] - 1s 1ms/step - loss: 0.3864 - accuracy: 0.8280
Epoch 4/50
1289/1289 [==============================] - 2s 1ms/step - loss: 0.3776 - accuracy: 0.8327
Epoch 5/50
1289/1289 [==============================] - 1s 1ms/step - loss: 0.3718 - accuracy: 0.8363
Epoch 6/50
1289/1289 [==============================] - 2s 1ms/step - loss: 0.3669 - accuracy: 0.8395
Epoch 7/50
1289/1289 [==============================] - 1s 1ms/step - loss: 0.3629 - accuracy: 0.8416
Epoch 8/50
1289/1289 [==============================] - 1s 892us/step - loss: 0.3597 - accuracy: 0.8439
Epoch 9/50
1289/1289 [==============================] - 2s 1ms/step - loss: 0.3574 - accuracy: 0.8440
Epoch 10/50
1289/1289 [==============================] - 1s 1ms/step - loss:

{'train_acc': 0.859426,
 'test_acc': 0.6325,
 'roc_auc': 0.721743,
 'class_0': {'precision': 0.945946, 'recall': 0.571249, 'f1': 0.712329},
 'class_1': {'precision': 0.342004, 'recall': 0.872236, 'f1': 0.491349},
 'ks_stats': 0.525584,
 'p_value': 0.0,
 'tp': 355,
 'tn': 910,
 'fp': 683,
 'fn': 52}

In [86]:

# Build the neural network
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=512, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Make predictions
predictions = model.predict(x_test)



# y_pred_train, y_pred_test, y_pred_test_proba = model_testing.prediction(model, x_train, x_test)

y_pred_train = np.round(model.predict(x_train))
y_pred_test = np.round(model.predict(x_test))
y_pred_test_proba = model.predict(x_test)

# Calculate evaluation metrics
model_evaluation = model_testing.Evaluation(y_train, y_test, y_pred_train, y_pred_test, y_pred_test_proba)

all_metrics,uhuh = model_evaluation.main()
uhuh

Epoch 1/10
26/26 [==============================] - 0s 2ms/step - loss: 0.6066 - accuracy: 0.6950
Epoch 2/10
26/26 [==============================] - 0s 1ms/step - loss: 0.4765 - accuracy: 0.7853
Epoch 3/10
26/26 [==============================] - 0s 1ms/step - loss: 0.4252 - accuracy: 0.8067
Epoch 4/10
26/26 [==============================] - 0s 1ms/step - loss: 0.4003 - accuracy: 0.8220
Epoch 5/10
26/26 [==============================] - 0s 1ms/step - loss: 0.3826 - accuracy: 0.8310
Epoch 6/10
26/26 [==============================] - 0s 1ms/step - loss: 0.3705 - accuracy: 0.8370
Epoch 7/10
26/26 [==============================] - 0s 2ms/step - loss: 0.3622 - accuracy: 0.8425
Epoch 8/10
26/26 [==============================] - 0s 2ms/step - loss: 0.3560 - accuracy: 0.8459
Epoch 9/10
26/26 [==============================] - 0s 1ms/step - loss: 0.3506 - accuracy: 0.8475
Epoch 10/10
26/26 [==============================] - 0s 1ms/step - loss: 0.3455 - accuracy: 0.8515
Test loss: 0.7091
T

{'train_acc': 0.852521,
 'test_acc': 0.672,
 'roc_auc': 0.733734,
 'class_0': {'precision': 0.93826, 'recall': 0.62963, 'f1': 0.753569},
 'class_1': {'precision': 0.366273, 'recall': 0.837838, 'f1': 0.509716},
 'ks_stats': 0.498111,
 'p_value': 0.0,
 'tp': 341,
 'tn': 1003,
 'fp': 590,
 'fn': 66}

In [16]:
# Build the neural network
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(x_train.shape[1],)))
model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Dense(4, activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dense(4, activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Make predictions
predictions = model.predict(x_test)


# y_pred_train, y_pred_test, y_pred_test_proba = model_testing.prediction(model, x_train, x_test)

y_pred_train = np.round(model.predict(x_train))
y_pred_test = np.round(model.predict(x_test))
y_pred_test_proba = model.predict(x_test)

# Calculate evaluation metrics
model_evaluation = model_testing.Evaluation(y_train, y_test, y_pred_train, y_pred_test, y_pred_test_proba)

all_metrics,uhuh = model_evaluation.main()
uhuh


Epoch 1/50
250/250 [==============================] - 1s 1ms/step - loss: 0.6354 - accuracy: 0.6179
Epoch 2/50
250/250 [==============================] - 0s 1ms/step - loss: 0.5151 - accuracy: 0.7615
Epoch 3/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4586 - accuracy: 0.8085
Epoch 4/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4330 - accuracy: 0.8148
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4208 - accuracy: 0.8206
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4130 - accuracy: 0.8213
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4075 - accuracy: 0.8241
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4011 - accuracy: 0.8311
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3942 - accuracy: 0.8319
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3924 - accuracy: 0.8338

{'train_acc': 0.858,
 'test_acc': 0.8585,
 'roc_auc': 0.699895,
 'class_0': {'precision': 0.869639, 'recall': 0.967357, 'f1': 0.915899},
 'class_1': {'precision': 0.77193, 'recall': 0.432432, 'f1': 0.554331},
 'ks_stats': 0.57089,
 'p_value': 0.0,
 'tp': 176,
 'tn': 1541,
 'fp': 52,
 'fn': 231}

In [19]:
# Build the neural network
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(100000, activation='tanh', input_shape=(x_train.shape[1],)))
model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Dense(4, activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dense(4, activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Make predictions
predictions = model.predict(x_test)


# y_pred_train, y_pred_test, y_pred_test_proba = model_testing.prediction(model, x_train, x_test)

y_pred_train = np.round(model.predict(x_train))
y_pred_test = np.round(model.predict(x_test))
y_pred_test_proba = model.predict(x_test)

# Calculate evaluation metrics
model_evaluation = model_testing.Evaluation(y_train, y_test, y_pred_train, y_pred_test, y_pred_test_proba)

all_metrics,uhuh = model_evaluation.main()
uhuh


Epoch 1/50
250/250 [==============================] - 10s 40ms/step - loss: 1.8610 - accuracy: 0.6289
Epoch 2/50
250/250 [==============================] - 11s 43ms/step - loss: 1.0027 - accuracy: 0.6817
Epoch 3/50
250/250 [==============================] - 10s 40ms/step - loss: 0.7639 - accuracy: 0.7092
Epoch 4/50
250/250 [==============================] - 10s 40ms/step - loss: 0.6255 - accuracy: 0.7412
Epoch 5/50
250/250 [==============================] - 10s 41ms/step - loss: 0.5556 - accuracy: 0.7690
Epoch 6/50
250/250 [==============================] - 11s 43ms/step - loss: 0.5232 - accuracy: 0.7807
Epoch 7/50
250/250 [==============================] - 11s 42ms/step - loss: 0.5032 - accuracy: 0.7884
Epoch 8/50
250/250 [==============================] - 11s 44ms/step - loss: 0.4831 - accuracy: 0.8041
Epoch 9/50
250/250 [==============================] - 11s 46ms/step - loss: 0.4680 - accuracy: 0.8031
Epoch 10/50
250/250 [==============================] - 12s 47ms/step - loss: 0.463

{'train_acc': 0.834,
 'test_acc': 0.8335,
 'roc_auc': 0.617433,
 'class_0': {'precision': 0.837259, 'recall': 0.981795, 'f1': 0.903785},
 'class_1': {'precision': 0.780303, 'recall': 0.253071, 'f1': 0.382189},
 'ks_stats': 0.498415,
 'p_value': 0.0,
 'tp': 103,
 'tn': 1564,
 'fp': 29,
 'fn': 304}

In [8]:
model.save_weights("./final_results/nn_1.h5")

In [9]:
existing_network_1 = tf.keras.models.Sequential()

existing_network_1.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(x_train.shape[1],)))
existing_network_1.add(tf.keras.layers.BatchNormalization())

existing_network_1.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
existing_network_1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

existing_network_1.load_weights('./final_results/nn_1.h5')

In [11]:
existing_network_2 = tf.keras.models.Sequential()

existing_network_2.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(x_train.shape[1],)))
existing_network_2.add(tf.keras.layers.BatchNormalization())

existing_network_2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
existing_network_2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

existing_network_2.load_weights('./final_results/nn_2.h5')

In [15]:
import tensorflow as tf

new_network = tf.keras.models.Sequential()

new_network.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(x_train.shape[1],)))
new_network.add(tf.keras.layers.BatchNormalization())

new_network.add(tf.keras.layers.Dense(1, activation='sigmoid'))


# # Load the weights from the existing networks
# existing_network_1 = tf.keras.models.load_model('./final_results/nn_1.h5')
# existing_network_2 = tf.keras.models.load_model('./final_results/nn_2.h5')

# Get the weights of each layer from the existing networks
weights_1 = existing_network_1.get_weights()
weights_2 = existing_network_2.get_weights()

# Combine the weights into the new network
combined_weights = [(w1) + (w2) for w1, w2 in zip(weights_1, weights_2)]
new_network.set_weights(combined_weights)

# Train the new network on your data

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

new_network.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# new_network.fit(x_train, y_train, epochs=1, batch_size=32, verbose=1)


# Evaluate the model
loss, accuracy = new_network.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Make predictions
predictions = new_network.predict(x_test)


# y_pred_train, y_pred_test, y_pred_test_proba = model_testing.prediction(model, x_train, x_test)

y_pred_train = np.round(new_network.predict(x_train))
y_pred_test = np.round(new_network.predict(x_test))
y_pred_test_proba = new_network.predict(x_test)

# Calculate evaluation metrics
model_evaluation = model_testing.Evaluation(y_train, y_test, y_pred_train, y_pred_test, y_pred_test_proba)

all_metrics,uhuh = model_evaluation.main()
uhuh


Test loss: 1.9542
Test accuracy: 0.5440
63/63 [==============================] - 0s 633us/step


{'train_acc': 0.5505,
 'test_acc': 0.544,
 'roc_auc': 0.655211,
 'class_0': {'precision': 0.92089, 'recall': 0.467671, 'f1': 0.620316},
 'class_1': {'precision': 0.287993, 'recall': 0.842752, 'f1': 0.429287},
 'ks_stats': 0.386305,
 'p_value': 0.0,
 'tp': 343,
 'tn': 745,
 'fp': 848,
 'fn': 64}

In [43]:
weights_1[0][1:].shape

(11, 11)